# import

In [40]:
import torch
import torch.nn as nn
from vilt.modules import heads, objectives
import vilt.modules.vision_transformer as vit
import torch.nn.functional as F
import random
from typing import OrderedDict
import os
import pandas as pd
import numpy as np
from vilt.transforms import pixelbert_transform
from PIL import Image
from torchvision import transforms
from tqdm import tqdm
from torch.utils.data import DataLoader
import gc
import torch.optim as optim
from torch.optim import lr_scheduler
from collections import defaultdict
import wandb
import pretrainedmodels
from sklearn.model_selection import StratifiedKFold, KFold, StratifiedGroupKFold


# config

In [41]:
class config:
    debug = False
    sensor_only = False
    crop_name = "total"

    
    exp_name = "vilt"
    seed = 101
    batch_size = 4096  # this is a desired batch size; pl trainer will accumulate gradients when per step batch is smaller.
    train_batch_size = 32
    valid_batch_size = 4
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    n_fold = 5

    model_name = "sensorViLOnlyTransformerSS" #仅图片
    # model_name = "sensorOnlyViLTransformerSS"  #仅vilt传感器
    # model_name = "sensorViLTransformerSS"  #vilt图像+传感器
    # model_name = "DNNF1"  #DNNF1图像+传感器
    # model_name = "DNNF1PictureOnly"  #DNNF1图像
    # model_name = "DNNF1SensorOnly"  #DNNF1传感器

    # model_name = "DNNF2"  #DNNF2图像+传感器
    # model_name = "DNNF2PictureOnly"  #DNNF1图像
    # model_name = "DNNF2SensorOnly"  #DNNF1传感器
    # wandb 
    # wandb_name = "vilt|大豆|290图像加传感器"
    # wandb_name = "vilt|大豆|290仅传感器"
    wandb_name = "vilt|大豆|290仅图片"

    # wandb_name = "DNNF1|大豆|290图像加传感器"
    # wandb_name = "DNNF1|大豆|290仅图像"
    # wandb_name = "DNNF1|大豆|290仅传感器"
    
    # wandb_name = "DNNF2|大豆|290图像加传感器"
    # wandb_name = "DNNF2|大豆|290仅图像"
    # wandb_name = "DNNF2|大豆|290仅传感器"
    

    # Image setting
    train_transform_keys = ["pixelbert"]
    val_transform_keys = ["pixelbert"]
    img_size = 384
    max_image_len = -1
    patch_size = 32
    draw_false_image = 1
    image_only = False

    # Sensor
    # senser_input_num = 11 # 翔冠的传感器参数
    senser_input_num = 19 # 天航的传感器参数
    
    # Text Setting
    vqav2_label_size = 3129
    max_text_len = 40
    tokenizer = "bert-base-uncased"
    vocab_size = 30522 # vocabulary词汇数量
    whole_word_masking = False
    mlm_prob = 0.15
    draw_false_text = 0

    # Transformer Setting
    vit = "vit_base_patch32_384"
    hidden_size = 768  # 嵌入向量大小
    num_heads = 12
    num_layers = 12
    mlp_ratio = 4
    drop_rate = 0.1

    # Optimizer Setting
    optim_type = "adamw"
    learning_rate = 1e-3 #0.0015#2e-3 #
    weight_decay = 1e-4 # 0.01 ->1e-4
    decay_power = 1
    max_epoch = 50
    max_steps = 25000
    warmup_steps = 2500
    end_lr = 0
    lr_mult = 1  # multiply lr for downstream heads
    # T_max = 8000/train_batch_size*max_epoch 
    # T_max = 4632/train_batch_size*max_epoch # total 7237.5
    T_max = 2126/train_batch_size*max_epoch # soybean 3321.875

    # Downstream Setting
    get_recall_metric = False


    # below params varies with the environment
    data_root = "/home/junsheng/data/ILSVR2012"
    log_dir = "result"
    per_gpu_batchsize = 0  # you should define this manually with per_gpu_batch_size=#
    num_gpus = 1
    num_nodes = 1
    load_path = "weights/vilt_200k_mlm_itm.ckpt"
    # load_path = "save_model_dict.pt"
    num_workers = 1
    precision = 16

# config = vars(config)
# config = dict(config)
config

if config.debug:
    config.max_epoch = 5

In [42]:
def setup_seed(seed):

    torch.manual_seed(seed)  # 为CPU设置随机种子
    np.random.seed(seed)  # Numpy module.
    random.seed(seed)  # Python random module.
    # torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.cuda.manual_seed(seed)  # 为当前GPU设置随机种子
    torch.cuda.manual_seed_all(seed)  # 为所有GPU设置随机种子
    #os.environ['PYTHONHASHSEED'] = str(seed)
    
setup_seed(config.seed)

# wandb

In [43]:
# os.environ["WANDB_MODE"] = 'dryrun' # 离线模式
try:
    # wandb.log(key="*******") # if debug
    wandb.login() # storage in ~/.netrc file
    anonymous = None
except:
    anonymous = "must"
    print('\nGet your W&B access token from here: https://wandb.ai/authorize\n')


# 数据

In [44]:
df = pd.read_csv("script/test_all.csv")
class_df = pd.read_csv("script/class_df.csv")
df['path'] = df.apply(lambda row:os.path.join("train",row.classification,row.filename),axis=1)
class_map = {}
for row_index,row in class_df.iterrows():
    class_map[row.classfication] = row['index']
df['classification_index'] = df.apply(lambda row:class_map[row.classification],axis=1)

In [45]:
df.head()

,classification,filename,path,classification_index
0,n02119789,n02119789_3374.JPEG,train/n02119789/n02119789_3374.JPEG,0
1,n02119789,n02119789_13567.JPEG,train/n02119789/n02119789_13567.JPEG,0
2,n02119789,n02119789_6483.JPEG,train/n02119789/n02119789_6483.JPEG,0
3,n02119789,n02119789_7609.JPEG,train/n02119789/n02119789_7609.JPEG,0
4,n02119789,n02119789_4810.JPEG,train/n02119789/n02119789_4810.JPEG,0


# dataset


In [46]:
myTransforms = transforms.Compose([
    transforms.Resize((config.img_size,config.img_size)),
    transforms.ToTensor(),
    transforms.Normalize(
    mean=[0.485, 0.456, 0.406], #imagenet的mean和std
    std=[0.229, 0.224, 0.225],
)
])

def load_img(path):
    img =  Image.open(path).convert('RGB')
    img = myTransforms(img)
    return img

class BuildDataset(torch.utils.data.Dataset):
    def __init__(self, df,transforms=None):
        self.df = df
        self.path  = df['path'].tolist()
        self.classification_index = df['classification_index'].tolist()
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path  = os.path.join(config.data_root,self.path[index])
        img = load_img(img_path)
        classification_index = self.classification_index[index]
        return torch.tensor(img).to(torch.float), torch.tensor(classification_index).to(torch.int)

# dataloader

In [50]:
def fetch_dataloader(df):


    train_data  = BuildDataset(df=df)
    # valid_data = BuildDataset(df=valid_df,label=True)

    train_loader = DataLoader(train_data, batch_size=config.train_batch_size,shuffle=True)
    # valid_loader = DataLoader(valid_data, batch_size=config.valid_batch_size,shuffle=False)
    # test_loader = DataLoader(test_data, batch_size=config.test_batch_size,shuffle=False)
    return train_loader

train_loader = fetch_dataloader(df)

In [52]:
imgs, labels = next(iter(train_loader))
print(imgs.shape,labels.shape)

torch.Size([32, 3, 384, 384]) torch.Size([32])


/tmp/ipykernel_693720/2557418770.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(img).to(torch.float), torch.tensor(classification_index).to(torch.int)


# model

## ViST预训练模型

In [ ]:
class ViST_pretrain(nn.Module):

    def __init__(self,sensor_class_n,output_class_n):
        super().__init__()
        # ####################################################prepare resnet
        resnet_model = pretrainedmodels.__dict__["resnet18"](
            num_classes=1000, pretrained='imagenet')
        features = list([resnet_model.conv1, resnet_model.bn1, resnet_model.relu, resnet_model.maxpool, resnet_model.layer1, resnet_model.layer2, resnet_model.layer3,resnet_model.layer4])

        conv = nn.Conv2d(512, 768, kernel_size=(1, 1), stride=(1, 1), bias=False)
        bn = nn.BatchNorm2d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        relu = nn.ReLU(inplace=True)
        self.resnet_features = nn.Sequential(*features,conv,bn,relu)
        ############################################################


        self.sensor_linear_pretrain = nn.Linear(144,145,bias=True)

        self.token_type_embeddings = nn.Embedding(2, config.hidden_size)
        self.token_type_embeddings.apply(objectives.init_weights)

        # self.transformer = getattr(vit, config.vit)(
        #         pretrained=False, config=vars(config)
        #     )
        self.transformer = vit.VisionTransformerForViST(img_size=config.img_size,patch_size=config.patch_size,embed_dim=config.hidden_size,num_heads=config.num_heads,mlp_ratio=config.mlp_ratio,qkv_bias=False,qk_scale=None)
        
       
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()


        self.pooler = heads.Pooler(config.hidden_size)

        # self.pooler.apply(objectives.init_weights)
        self.classifier = nn.Linear(config.hidden_size,output_class_n)

        hs = config.hidden_size


    def infer(
        self,
        batch,
        mask_image=False,
        image_token_type_idx=1,
        image_embeds=None,
        image_masks=None,
    ):
        sensor = batch['sensor'].to(config.device) 
        sensor_embeds = self.resnet_features(sensor).flatten(2)
        sensor_embeds = self.sensor_linear_pretrain(sensor_embeds)
        sensor_embeds = torch.transpose(sensor_embeds,1,2)

        if image_embeds is None and image_masks is None:
            img = batch["image"].to(config.device)
            (
                image_embeds, # torch.Size([1, 217, 768])
                image_masks, # torch.Size([1, 217])
                patch_index,
                image_labels,
            ) = self.transformer.visual_embed(
                img,
                max_image_len=config.max_image_len,
                mask_it=mask_image,
            )
        else:
            patch_index, image_labels = (
                None,
                None,
            )
        # 用embedding对数据输入预处理，降低维度
        image_embeds = image_embeds + self.token_type_embeddings(
                torch.full_like(image_masks, image_token_type_idx)
            )
        
        x_image = image_embeds # torch.Size([2, 145, 768])
        x_sensor = sensor_embeds # torch.Size([2, 145, 768])
        for i, blk in enumerate(self.transformer.blocks): 
            blk = blk.to(config.device)
            x_image,x_sensor, _attn_image,_attn_sensor = blk(x_image, x_sensor) # co_masks = torch.Size([1, 211])

        x = torch.cat([x_image, x_sensor], dim=1)
        x = self.transformer.norm(x) # torch.Size([1, 240, 768])

        cls_feats = self.pooler(x) # torch.Size([1, 768])
        # cls_feats = self.dense(x)
        # cls_feats = self.activation(cls_feats)
        cls_output = self.classifier(cls_feats)
        m = nn.Softmax(dim=1)
        
        # m = nn.Sigmoid()
        cls_output = m(cls_output)
        
        
        ret = {
           "sensor_feats":x_image,
            "image_feats": x_sensor,
            "cls_feats": cls_feats, # class features
            "cls_output":cls_output,
        }

        return ret

    def forward(self, batch):
        ret = dict()
        
        ret.update(self.infer(batch))
        return ret

# 损失函数

In [ ]:
criterion = F.mse_loss #均方误差损失函数
criterion_mae = nn.L1Loss()

# train one epoch

In [ ]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    dataset_size = 0
    running_loss = 0.0
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Train ')
    for step, (img, sensor,label) in pbar:         
        # img = img.to(device, dtype=torch.float)
        # sensor  = sensor.to(device, dtype=torch.float)
        # label  = label.to(device, dtype=torch.float)
        batch_size = img.size(0)
        
        batch = {"image":img,"sensor":sensor}

        y_pred = model(batch)
        label = label.to(config.device).unsqueeze(1)
        loss = criterion(y_pred['cls_output'], label)
        
        #一坨优化
        optimizer.zero_grad()#每一次反向传播之前都要归零梯度
        loss.backward()      #反向传播
        optimizer.step()     #固定写法
        scheduler.step()
     
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        epoch_loss = running_loss / dataset_size
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(train_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_mem=f'{mem:0.2f} GB')

    
        
        
    torch.cuda.empty_cache()
    gc.collect()
    
    return epoch_loss

# valid one epoch

In [ ]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, optimizer):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    running_loss_mae = 0.0
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Valid ')
    for step, (img, sensor,label) in pbar:               
        
        
        batch_size = img.size(0)
        batch = {"image":img,"sensor":sensor}

        y_pred  = model(batch)
        label = label.to(config.device).unsqueeze(1)

        loss = criterion(y_pred['cls_output'], label)
        loss_mae = criterion_mae(y_pred['cls_output'], label)

        running_loss += (loss.item() * batch_size)
        running_loss_mae += (loss_mae.item() * batch_size)

        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        epoch_loss_mae = running_loss_mae / dataset_size
        
        
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(valid_loss=f'{epoch_loss:0.4f}',
        valid_loss_mae=f'{epoch_loss_mae:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_memory=f'{mem:0.2f} GB')
    torch.cuda.empty_cache()
    gc.collect()
    
    return epoch_loss,epoch_loss_mae#MSE，MAE

# train

In [ ]:

def run_training(model, optimizer, scheduler, device, num_epochs,train_loader,valid_loader):
     # init wandb
    run = wandb.init(project="vilt",
                    config={k: v for k, v in dict(vars(config)).items() if '__' not in k},
                    # config={k: v for k, v in dict(config).items() if '__' not in k},
                    anonymous=anonymous,
                    # name=f"vilt|fold-{config.valid_fold}",
                    name=config.wandb_name,
                    # group=config.wandb_group,
                    )
    wandb.watch(model, log_freq=100)

    best_loss = 9999
    best_valid_loss = 9999
    history = defaultdict(list)
    if torch.cuda.is_available():
        print("cuda: {}\n".format(torch.cuda.get_device_name()))
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        print(f'Epoch {epoch}/{num_epochs}', end='')
        train_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=device, epoch=epoch)
        val_loss,val_loss_mae = valid_one_epoch(model,valid_loader,device=device,optimizer=optimizer)
        history['Train Loss'].append(train_loss)
        history['Valid Loss'].append(val_loss)
        history['Valid Loss MAE'].append(val_loss_mae)

        wandb.log({"Train Loss": train_loss,
                    "Valid Loss": val_loss,
                    "Valid Loss MAE": val_loss_mae,
                "lr": scheduler.get_last_lr()[0]
                })
        if best_valid_loss > val_loss:
            best_valid_loss = val_loss
            # model_file_path = os.path.join(wandb.run.dir,"epoch-{}-{}.bin".format(epoch,wandb.run.id))
            model_file_path = os.path.join(wandb.run.dir,"epoch-best.bin")
            run.summary["Best Epoch"] = epoch
            torch.save(model.state_dict(), model_file_path)
            print("model save to", model_file_path)
               
    os.system("cp /home/junsheng/ViLT/my_vilt_tianhang_soybean.ipynb {}".format(wandb.run.dir))
    run.finish()
    return model, history

run train

In [ ]:
def run(crop_name:str,model_name:str,wandb_name:str,sensor_only:bool):
    config.model_name = model_name
    config.wandb_name = wandb_name
    config.sensor_only = sensor_only

    df = fetch_df(crop_name)
    df = creat_folds(df)

    train_loader,valid_loader = fetch_dataloader(fold=0,df=df)
    # train_loader,valid_loader = fetch_dataloader_ubiquatous()


    model = build_model(config.model_name,True)
    model.to(config.device)
    print(config.device)
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=config.T_max, 
                                                    eta_min=5e-8)
    model, history = run_training(model, optimizer, scheduler,device=config.device,num_epochs=config.max_epoch,train_loader=train_loader,valid_loader=valid_loader)

## tasks

soybean task

In [ ]:
soybean_task = [
    # ********************vist*****************
    {
        "crop_name":"soybean",
        "model_name":"ViST",
        "wandb_name":"vist|大豆|290图像加传感器",
        "sensor_only":False,
        
    },
    {
        "crop_name":"soybean",
        "model_name":"imageViST",
        "wandb_name":"vist|大豆|290仅图片",
        "sensor_only":True,
    },
    {
        "crop_name":"soybean",
        "model_name":"sensorViST",
        "wandb_name":"vist|大豆|290仅传感器",
        "sensor_only":True,
    },
    # ********************vist2***************** self attn和cross attn混用
    {
        "crop_name":"soybean",
        "model_name":"ViST2",
        "wandb_name":"vist|大豆|290图像加传感器",
        "sensor_only":False,
        
    },

    # ********************vilt*****************
    {
        "crop_name":"soybean",
        "model_name":"sensorViLOnlyTransformerSS",
        "wandb_name":"vilt|大豆|290仅图片",
        "sensor_only":True,
    },
    {
        "crop_name":"soybean",
        "model_name":"sensorOnlyViLTransformerSS",
        "wandb_name":"vilt|大豆|290仅传感器",
        "sensor_only":True,
    },
    {
        "crop_name":"soybean",
        "model_name":"sensorViLTransformerSS",
        "wandb_name":"vilt|大豆|290图像加传感器",
        "sensor_only":False,
    },

    # ********************DNNF1*****************
    {
        "crop_name":"soybean",
        "model_name":"DNNF1",
        "wandb_name":"DNNF1|大豆|290图像加传感器",
        "sensor_only":False,
    },
    {
        "crop_name":"soybean",
        "model_name":"DNNF1PictureOnly",
        "wandb_name":"DNNF1|大豆|290仅图像",
        "sensor_only":True,
    },
    {
        "crop_name":"soybean",
        "model_name":"DNNF1SensorOnly",
        "wandb_name":"DNNF1|大豆|290仅传感器",
        "sensor_only":True,
    },
    # ********************DNNF2*****************
    {
        "crop_name":"soybean",
        "model_name":"DNNF2",
        "wandb_name":"DNNF2|大豆|290图像加传感器",
        "sensor_only":False,
    },
    {
        "crop_name":"soybean",
        "model_name":"DNNF2PictureOnly",
        "wandb_name":"DNNF2|大豆|290仅图像",
        "sensor_only":True,
    },
    {
        "crop_name":"soybean",
        "model_name":"DNNF2SensorOnly",
        "wandb_name":"DNNF2|大豆|290仅传感器",
        "sensor_only":True,
    },

]


corn task

In [ ]:

corn_task = [
    # ********************vist*****************
    {
        "crop_name":"corn",
        "model_name":"ViST",
        "wandb_name":"vist|玉米|290图像加传感器",
        "sensor_only":False,
    },
    {
        "crop_name":"corn",
        "model_name":"imageViST",
        "wandb_name":"vist|玉米|290仅图片",
        "sensor_only":True,
    },
    {
        "crop_name":"corn",
        "model_name":"sensorViST",
        "wandb_name":"vist|玉米|290仅传感器",
        "sensor_only":True,
    },
    # ********************vist2*****************
    {
        "crop_name":"corn",
        "model_name":"ViST2",
        "wandb_name":"vist|玉米|290图像加传感器",
        "sensor_only":False,
    },
    # ********************vilt*****************
    {
        "crop_name":"corn",
        "model_name":"sensorViLOnlyTransformerSS",
        "wandb_name":"vilt|玉米|290仅图片",
        "sensor_only":True,
    },
    {
        "crop_name":"corn",
        "model_name":"sensorOnlyViLTransformerSS",
        "wandb_name":"vilt|玉米|290仅传感器",
        "sensor_only":True,
    },
    {
        "crop_name":"corn",
        "model_name":"sensorViLTransformerSS",
        "wandb_name":"vilt|玉米|290图像加传感器",
        "sensor_only":False,
    },

    # ********************DNNF1*****************
    {
        "crop_name":"corn",
        "model_name":"DNNF1",
        "wandb_name":"DNNF1|玉米|290图像加传感器",
        "sensor_only":False,
    },
    {
        "crop_name":"corn",
        "model_name":"DNNF1PictureOnly",
        "wandb_name":"DNNF1|玉米|290仅图像",
        "sensor_only":True,
    },
    {
        "crop_name":"corn",
        "model_name":"DNNF1SensorOnly",
        "wandb_name":"DNNF1|玉米|290仅传感器",
        "sensor_only":True,
    },
    # ********************DNNF2*****************
    {
        "crop_name":"corn",
        "model_name":"DNNF2",
        "wandb_name":"DNNF2|玉米|290图像加传感器",
        "sensor_only":False,
    },
    {
        "crop_name":"corn",
        "model_name":"DNNF2PictureOnly",
        "wandb_name":"DNNF2|玉米|290仅图像",
        "sensor_only":True,
    },
    {
        "crop_name":"corn",
        "model_name":"DNNF2SensorOnly",
        "wandb_name":"DNNF2|玉米|290仅传感器",
        "sensor_only":True,
    },

]


rice

In [ ]:
rice_task = [
    # ********************vist*****************
    {
        "crop_name":"rice",
        "model_name":"ViST",
        "wandb_name":"vist|水稻|290图像加传感器",
        "sensor_only":False,
    },
    {
        "crop_name":"rice",
        "model_name":"imageViST",
        "wandb_name":"vist|水稻|290仅图片",
        "sensor_only":True,
    },
    {
        "crop_name":"rice",
        "model_name":"sensorViST",
        "wandb_name":"vist|水稻|290仅传感器",
        "sensor_only":True,
    },
    # ********************vist2*****************
    {
        "crop_name":"rice",
        "model_name":"ViST2",
        "wandb_name":"vist|水稻|290图像加传感器",
        "sensor_only":False,
    },
    # ********************vilt*****************
    {
        "crop_name":"rice",
        "model_name":"sensorViLOnlyTransformerSS",
        "wandb_name":"vilt|水稻|290仅图片",
        "sensor_only":True,
    },
    {
        "crop_name":"rice",
        "model_name":"sensorOnlyViLTransformerSS",
        "wandb_name":"vilt|水稻|290仅传感器",
        "sensor_only":True,
    },
    {
        "crop_name":"rice",
        "model_name":"sensorViLTransformerSS",
        "wandb_name":"vilt|水稻|290图像加传感器",
        "sensor_only":False,
    },

    # ********************DNNF1*****************
    {
        "crop_name":"rice",
        "model_name":"DNNF1",
        "wandb_name":"DNNF1|水稻|290图像加传感器",
        "sensor_only":False,
    },
    {
        "crop_name":"rice",
        "model_name":"DNNF1PictureOnly",
        "wandb_name":"DNNF1|水稻|290仅图像",
        "sensor_only":True,
    },
    {
        "crop_name":"rice",
        "model_name":"DNNF1SensorOnly",
        "wandb_name":"DNNF1|水稻|290仅传感器",
        "sensor_only":True,
    },
    # ********************DNNF2*****************
    {
        "crop_name":"rice",
        "model_name":"DNNF2",
        "wandb_name":"DNNF2|水稻|290图像加传感器",
        "sensor_only":False,
    },
    {
        "crop_name":"rice",
        "model_name":"DNNF2PictureOnly",
        "wandb_name":"DNNF2|水稻|290仅图像",
        "sensor_only":True,
    },
    {
        "crop_name":"rice",
        "model_name":"DNNF2SensorOnly",
        "wandb_name":"DNNF2|水稻|290仅传感器",
        "sensor_only":True,
    },

]


total task

In [ ]:
total_task = [
    # ********************vist*****************
    {
        "crop_name":"total",
        "model_name":"ViST",
        "wandb_name":"vist|普适|290图像加传感器",
        "sensor_only":False,
    },
    {
        "crop_name":"total",
        "model_name":"imageViST",
        "wandb_name":"vist|普适|290仅图片",
        "sensor_only":True,
    },
    {
        "crop_name":"total",
        "model_name":"sensorViST",
        "wandb_name":"vist|普适|290仅传感器",
        "sensor_only":True,
    },
   # ********************vilt*****************
    {
        "crop_name":"total",
        "model_name":"sensorViLOnlyTransformerSS",
        "wandb_name":"vilt|普适|290仅图片",
        "sensor_only":True,
    },
    {
        "crop_name":"total",
        "model_name":"sensorOnlyViLTransformerSS",
        "wandb_name":"vilt|普适|290仅传感器",
        "sensor_only":True,
    },
    {
        "crop_name":"total",
        "model_name":"sensorViLTransformerSS",
        "wandb_name":"vilt|普适|290图像加传感器",
        "sensor_only":False,
    },

    # ********************DNNF1*****************
    {
        "crop_name":"total",
        "model_name":"DNNF1",
        "wandb_name":"DNNF1|普适|290图像加传感器",
        "sensor_only":False,
    },
    {
        "crop_name":"total",
        "model_name":"DNNF1PictureOnly",
        "wandb_name":"DNNF1|普适|290仅图像",
        "sensor_only":True,
    },
    {
        "crop_name":"total",
        "model_name":"DNNF1SensorOnly",
        "wandb_name":"DNNF1|普适|290仅传感器",
        "sensor_only":True,
    },
    # ********************DNNF2*****************
    {
        "crop_name":"total",
        "model_name":"DNNF2",
        "wandb_name":"DNNF2|普适|290图像加传感器",
        "sensor_only":False,
    },
    {
        "crop_name":"total",
        "model_name":"DNNF2PictureOnly",
        "wandb_name":"DNNF2|普适|290仅图像",
        "sensor_only":True,
    },
    {
        "crop_name":"total",
        "model_name":"DNNF2SensorOnly",
        "wandb_name":"DNNF2|普适|290仅传感器",
        "sensor_only":True,
    },


]

run tast

In [ ]:
# 用两个预测另一个
# config.senser_input_num = 17
# ubiquatous_task =     {
#         "crop_name":"total",
#         "model_name":"sensorViLTransformerSS",
#         "wandb_name":"vilt|普适|290图像加传感器->水稻",
#         "sensor_only":False,
#     }
# config.learning_rate = 5e-5
# run(ubiquatous_task["crop_name"],ubiquatous_task["model_name"],ubiquatous_task["wandb_name"],ubiquatous_task["sensor_only"])

In [ ]:
# for i in range(3): # 0 1 2
# for i in [6,7,8]: # 0 1 2
#     config.learning_rate = 5e-5
#     run(total_task[i]["crop_name"],total_task[i]["model_name"],total_task[i]["wandb_name"],total_task[i]["sensor_only"])
# config.learning_rate = 5e-5
config.learning_rate = 0.0001
for i in [3]:
    task = corn_task[i]
    run(task["crop_name"],task["model_name"],task["wandb_name"],task["sensor_only"])
    task = rice_task[i]
    run(task["crop_name"],task["model_name"],task["wandb_name"],task["sensor_only"])
    task = soybean_task[i]
    run(task["crop_name"],task["model_name"],task["wandb_name"],task["sensor_only"])

# for i in [0,1,2]:
#     task = corn_task[i]
#     run(task["crop_name"],task["model_name"],task["wandb_name"],task["sensor_only"])

    

#  test


In [ ]:
def test():
    model = build_model("ViST",True)
    state_dict = torch.load('/home/junsheng/ViLT/wandb/run-20230111_141431-tb52bngc/files/epoch-best.bin')
    model.load_state_dict(state_dict)
    model.to(config.device)
    test_df = pd.read_csv("/home/junsheng/ViLT/data/ubiquitous_soybean.csv")
    tianhang_sensor = ['co2', 'stemp', 'stemp2', 'stemp3', 'stemp5', 'shumi', 'shumi2', 'shumi3', 'shumi5', 'humi', 'pm10', 'pm25', 'press', 'solar', 'temp', 'wind_d', 'wind_sp']
    test_df['sensor'] = test_df[tianhang_sensor].values.tolist()
    test_data = BuildDataset(df=test_df,label=True)
    test_loader = DataLoader(test_data, batch_size=config.valid_batch_size,shuffle=False)
    val_loss,val_loss_mae = valid_one_epoch(model,test_loader,device=config.device)